In [1]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import random
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

2025-10-01 16:08:44.382785: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-01 16:08:44.397386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-01 16:08:44.414197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-01 16:08:44.418938: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 16:08:44.432868: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# List all physical devices
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:")
for gpu in gpus:
    print(f" - {gpu.name}")

Available GPUs:
 - /physical_device:GPU:0


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPUs.")
    except RuntimeError as e:
        print(e)

Memory growth enabled for GPUs.


In [5]:
traffic_data = pd.read_csv('GD030A_S.csv')

## 1. Recover timestamp

In [6]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [7]:
# Apply the recover_timestamp function to recover the full time series
traffic_full = recover_timestamp(traffic_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


## 2. Train, validate, test data split

In [8]:
train_set = traffic_full[:'2022-02-28 23:00:00']
valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
test_set = traffic_full['2023-01-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full)))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full)))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full)))

Proportion of train_set : 0.6037
Proportion of valid_set : 0.2094
Proportion of test_set : 0.1869


In [9]:
print(train_set.isnull().sum(), len(train_set))
print(valid_set.isnull().sum(),len(valid_set))
print(test_set.isnull().sum(),len(test_set))

date    927
time    927
flow    927
dtype: int64 21168
date    91
time    91
flow    91
dtype: int64 7344
date    403
time    403
flow    403
dtype: int64 6552


## 3. Normalise the data 

In [10]:
# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data's 'flow' feature
scaler.fit(train_set[['flow']])

# Transform the 'flow' feature in all datasets
train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
test_set.loc[:, 'flow_scaled'] = scaler.transform(test_set[['flow']])

/tmp/ipykernel_67607/1100395100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
/tmp/ipykernel_67607/1100395100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
/tmp/ipykernel_67607/1100395100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

## 4. Split the data into X and y

In [11]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates input-output sequences for time series data, excluding any sequences containing NaN values.
    
    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow_scaled' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.
    
    Returns:
    - X: numpy array of shape (num_valid_samples, input_length, num_features)
    - y: numpy array of shape (num_valid_samples, forecast_horizon)
    """
    X, y = [], []
    num_features = data.shape[1]
    total_length = input_length + forecast_horizon
    
    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow_scaled'].values
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow_scaled'].values
        
        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            X.append(X_seq)
            y.append(y_seq)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs
        
    # Convert to numpy arrays and reshape X to match LSTM expected input (samples, timesteps, features)
    X = np.array(X).reshape(-1, input_length, 1)
    y = np.array(y).reshape(-1, forecast_horizon)
    
    return X, y

## 5. Create X and y

In [12]:
# Define Input Sequence Lengths
input_lengths = [24 * i for i in range(1, 22)]  # [24, 48, ..., 168]

In [13]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")
    
    # Create sequences with forecast_horizon=6
    X_train, y_train = create_sequences(train_set, length, forecast_horizon=6)
    X_val, y_val = create_sequences(valid_set, length, forecast_horizon=6)
    X_test, y_test = create_sequences(test_set, length, forecast_horizon=6)
    
    # Store in the dictionary
    data_dict[length]['X_train'] = X_train
    data_dict[length]['y_train'] = y_train
    data_dict[length]['X_val'] = X_val
    data_dict[length]['y_val'] = y_val
    data_dict[length]['X_test'] = X_test
    data_dict[length]['y_test'] = y_test
    
    # Print shapes and ensure no NaNs
    print(f"  X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"  X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
    print(f"  X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")

Processing input length: 24
  X_train shape: (19806, 24, 1), y_train shape: (19806, 6)
  X_val shape: (7108, 24, 1), y_val shape: (7108, 6)
  X_test shape: (5915, 24, 1), y_test shape: (5915, 6)

Processing input length: 48
  X_train shape: (19446, 48, 1), y_train shape: (19446, 6)
  X_val shape: (6988, 48, 1), y_val shape: (6988, 6)
  X_test shape: (5728, 48, 1), y_test shape: (5728, 6)

Processing input length: 72
  X_train shape: (19086, 72, 1), y_train shape: (19086, 6)
  X_val shape: (6868, 72, 1), y_val shape: (6868, 6)
  X_test shape: (5560, 72, 1), y_test shape: (5560, 6)

Processing input length: 96
  X_train shape: (18726, 96, 1), y_train shape: (18726, 6)
  X_val shape: (6748, 96, 1), y_val shape: (6748, 6)
  X_test shape: (5392, 96, 1), y_test shape: (5392, 6)

Processing input length: 120
  X_train shape: (18366, 120, 1), y_train shape: (18366, 6)
  X_val shape: (6628, 120, 1), y_val shape: (6628, 6)
  X_test shape: (5224, 120, 1), y_test shape: (5224, 6)

Processing input

## 6. Build LSTM model

In [14]:
def build_improved_lstm_model(hyperparams, input_length):
    """
    Builds an improved LSTM model based on provided hyperparameters and input length.

    Parameters:
    - hyperparams: dict containing 'units', 'dropout', 'learning_rate', and optionally 'recurrent_dropout'.
    - input_length: int, length of the input sequences.

    Returns:
    - model: compiled Keras model ready for training.
    """
    model = Sequential()
    
    # First LSTM layer with return_sequences=True to stack another LSTM layer
    model.add(LSTM(
        units=hyperparams['units'], 
        activation='tanh', 
        input_shape=(input_length, 1), 
        return_sequences=True, 
        #recurrent_dropout=hyperparams.get('recurrent_dropout', 0.0)
    ))
    model.add(Dropout(rate=hyperparams['dropout']))
    
    # Second LSTM layer
    model.add(LSTM(
        units=hyperparams['units'], 
        activation='tanh',
        #recurrent_dropout=hyperparams.get('recurrent_dropout', 0.0)
    ))
    model.add(Dropout(rate=hyperparams['dropout']))
    
    # Output layer for multi-step forecasting
    model.add(Dense(6))
    
    # Compile the model with MSE as the loss function
    optimizer = keras.optimizers.Adam(learning_rate=hyperparams['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model


## 7. Retrain the model after getting the best hyperparameters of each input length

In [15]:
best_hyperparameters_improved_model_lstm = {
    24: {
        'units': 64,
        'dropout': 0.2,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    48: {
        'units': 256,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    72: {
        'units': 256,
        'dropout': 0.1,
        'learning_rate': 0.001,
        'batch_size': 128
    },
    96: {
        'units': 256,
        'dropout': 0.4,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    120: {
        'units': 128,
        'dropout': 0.1,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    144: {
        'units': 256,
        'dropout': 0.2,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    168: {
        'units': 256,
        'dropout': 0.3,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    192: {
        'units': 256,
        'dropout': 0.4,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    216: {
        'units': 256,
        'dropout': 0.4,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    240: {
        'units': 64,
        'dropout': 0.4,
        'learning_rate': 0.005,
        'batch_size': 64
    },
    264: {
        'units': 256,
        'dropout': 0.4,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    288: {
        'units': 64,
        'dropout': 0.3,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    312: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    336: {
        'units': 256,
        'dropout': 0.1,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    360: {
        'units': 32,
        'dropout': 0.1,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    384: {
        'units': 256,
        'dropout': 0.3,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    408: {
        'units': 256,
        'dropout': 0.4,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    432: {
        'units': 128,
        'dropout': 0.4,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    456: {
        'units': 128,
        'dropout': 0.0,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    480: {
        'units': 256,
        'dropout': 0.1,
        'learning_rate': 0.001,
        'batch_size': 64
    },
    504: {
        'units': 128,
        'dropout': 0.0,
        'learning_rate': 0.001,
        'batch_size': 32
    }
}


In [2]:
import logging

logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format='%(asctime)s - %(message)s',  # Customize the log message format
    handlers=[
        logging.FileHandler('lstm2.log'),  # Log messages to 'output.log'
        logging.StreamHandler()             # Also output to console/notebook
    ]
)

In [16]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [17]:
from tensorflow.keras.callbacks import Callback

# Custom callback to record time per epoch
class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.times = []
    
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()
    
    def on_epoch_end(self, epoch, logs=None):
        self.times.append(time.time() - self.epoch_time_start)

In [18]:
def train_model(hyperparams,data_dict,length, seed=None):  # add seed
    if seed is not None:
        set_seed(seed)
    
    #get the data of each length
    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    
    # Train the model
    #model = build_lstm_model(hyperparams, length)    
    model = build_improved_lstm_model(hyperparams, length)
    # Early Stopping Callback
    early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True,verbose=1)   
    
    # Time history callback to record training time per epoch
    time_callback = TimeHistory()
    
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=hyperparams['batch_size'],
        validation_data=(X_val, y_val),
        callbacks=[early_stop, time_callback],
        verbose=0  # Set to 1 to see training progress
    )
    
    # Retrieve the best validation MSE from the history
    best_mse = min(history.history['val_loss'])
    logging.info(f"Validation loss: {best_mse:.5f}")

    # Calculate the average training time per epoch based on the actual epochs run
    avg_epoch_time = sum(time_callback.times) / len(time_callback.times)
    logging.info(f"Average training time per epoch: {avg_epoch_time:.6f} seconds over {len(time_callback.times)} epochs")
    
    return model, best_mse, avg_epoch_time

In [19]:
# Make predictions
def make_prediction(model, X_obs, y_obs):
    # Record start time
    start_time = time.time()
    
    y_pred = model.predict(X_obs,verbose=0)

    # Record end time and compute the inference time
    inference_time = time.time() - start_time
    logging.info(f"\nInference time: {inference_time:.6f} seconds")
    
    n_samples = X_obs.shape[0]
    output_len = y_obs.shape[1]

    # Reshape for inverse scaling
    y_pred_reshaped = y_pred.reshape(-1, 1)
    y_obs_reshaped = y_obs.reshape(-1, 1)

    # Inverse transform
    y_pred_inverse = scaler.inverse_transform(y_pred_reshaped).reshape(n_samples, output_len)
    y_obs_inverse = scaler.inverse_transform(y_obs_reshaped).reshape(n_samples, output_len)

    return y_pred_inverse, y_obs_inverse, inference_time

In [20]:
# Compute Metrics for Each Time Step
def evaluation(y_pred_inverse, y_obs_inverse):
    
    output_len = y_pred_inverse.shape[1]
    metrics_list = []  # To store metrics for each time step
    
    for i in range(output_len):
        y_true = y_obs_inverse[:, i]
        y_pred = y_pred_inverse[:, i]

        # Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_true, y_pred)

        # Mean Squared Error (MSE)
        mse = mean_squared_error(y_true, y_pred)

        # Root Mean Squared Error (RMSE)
        rmse = np.sqrt(mse)

        # Mean Absolute Percentage Error (MAPE)
        # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
        epsilon = 1e-10
        y_true_safe = np.where(y_true == 0, epsilon, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

                # Append the metrics for the current time step to the list
        metrics_list.append({
            'Time Step': i + 1,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE (%)': mape
        })

    # Create a DataFrame from the list of metrics
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.set_index('Time Step', inplace=True)

    return metrics_df

In [ ]:
mean_metrics_list=[]
#for length in best_hyperparameters_improved_model.keys():
for length in best_hyperparameters_improved_model_lstm.keys():
    logging.info(length)
    
    # Number of runs
    n_runs = 10
    
    # get the best hyperparameter of each length
    #hyperparams = best_hyperparameters_improved_model[length]
    hyperparams = best_hyperparameters_improved_model_lstm[length]
    
    # Initialize lists to store mean metrics and all metrics from each run
    mean_metrics_rows = []
    df_all_metrics_list = []
    successful_runs = []  # To track runs with valid mse

    for run in range(1, n_runs + 1):
        logging.info(f"\n--- Run {run} ---")

        # Optionally set a unique seed for each run to ensure variability
        seed = run
        # Train the model
        model, mse = train_model(hyperparams, data_dict, length, seed=seed)
        
        if not np.isnan(mse):  # Only proceed if mse is valid
            successful_runs.append(run)
        
            X_train = data_dict[length]['X_train']
            y_train = data_dict[length]['y_train']
            X_val = data_dict[length]['X_val']
            y_val = data_dict[length]['y_val']
            X_test = data_dict[length]['X_test']
            y_test = data_dict[length]['y_test']

            #get the true flow and predicted flow
            y_pred_train, y_obs_train = make_prediction(model, X_train, y_train)
            y_pred_val, y_obs_val = make_prediction(model, X_val, y_val)
            y_pred_test, y_obs_test = make_prediction(model, X_test, y_test)

            #calculate the evaluation metrics of each output step
            df_train = evaluation(y_pred_train, y_obs_train).add_suffix('_train')
            df_val = evaluation(y_pred_val, y_obs_val).add_suffix('_val')
            df_test = evaluation(y_pred_test, y_obs_test).add_suffix('_test')

            df_all_metrics = pd.concat([df_train, df_val, df_test], axis=1)
            df_all_metrics.index.name = length

            # Append df_all_metrics to the list
            df_all_metrics_list.append(df_all_metrics)

            # Calculate mean for all output step
            #mean_metrics = df_val.mean()
            mean_metrics = pd.concat([df_train.mean(), df_val.mean(), df_test.mean()])
            mean_metrics_row = pd.DataFrame(mean_metrics).T
            mean_metrics_row['MSE_val(loss)'] = mse
            mean_metrics_row['input_len'] = length
            #mean_metrics_row = mean_metrics_row[['input_len','MSE_val(loss)', 'MAE_val', 'RMSE_val', 'MAPE (%)_val']]
            mean_metrics_row = mean_metrics_row[['input_len', 'MSE_val(loss)', 'MAE_train', 'RMSE_train', 'MAPE (%)_train',
                                             'MAE_val', 'RMSE_val', 'MAPE (%)_val',
                                             'MAE_test', 'RMSE_test', 'MAPE (%)_test']]

            # Append to the list
            mean_metrics_rows.append(mean_metrics_row)
        else:
            logging.info(f"Run {run} has mse=NaN, skipping.")
            
    # Check if there are successful runs before proceeding
    if successful_runs:       
        # Concatenate all df_all_metrics into a single DataFrame with a new level for runs
        concatenated_all_metrics = pd.concat(df_all_metrics_list, keys=successful_runs, names=['Run', 'Time Step'])

        # Calculate the mean across runs for each metric and time step
        # This will group by 'Time Step' and calculate the mean of each metric across all runs
        aggregated_all_metrics_mean = concatenated_all_metrics.groupby('Time Step').mean()
        aggregated_all_metrics_mean.index.name = length

        logging.info("\n--- Aggregated Mean of All Metrics Across 10(Successful) Runs ---")
        # Convert DataFrame to string
        aggregated_all_metrics_mean_str = aggregated_all_metrics_mean.to_string(index=False)    
        # Log the DataFrame
        logging.info("\n" + aggregated_all_metrics_mean_str)
        display(aggregated_all_metrics_mean)
    else:
        logging.info(f"No successful runs for input length {length}.")

    # Check if mean metrics were calculated
    if mean_metrics_rows:        
        # After all runs, create a DataFrame of mean metrics
        mean_metrics_df = pd.concat(mean_metrics_rows, ignore_index=True)
        # Calculate the mean of each metric across the 10 runs
        final_mean_metrics = mean_metrics_df.mean()

        # Create a DataFrame for the final mean metrics
        final_mean_metrics_df = pd.DataFrame(final_mean_metrics).T

        mean_metrics_list.append(final_mean_metrics_df)
    else:
        logging.info(f"No mean metrics calculated for input length {length}.")

mean_metrics_df = pd.concat(mean_metrics_list).reset_index(drop=True)
logging.info("\n--- Final Mean Metrics Across 10 Runs ---")
# Convert DataFrame to string
mean_metrics_df_str = mean_metrics_df.to_string(index=False)    
# Log the DataFrame
logging.info("\n" + mean_metrics_df_str)
mean_metrics_df

2025-01-09 04:18:04,995 - 24
2025-01-09 04:18:04,996 - 
--- Run 1 ---
2025-01-09 04:18:05.202507: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18612 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 276f:00:00.0, compute capability: 8.0
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-01-09 04:18:08.277144: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


Epoch 72: early stopping
Restoring model weights from the end of the best epoch: 52.


2025-01-09 04:26:46,176 - Validation loss: 0.00593
2025-01-09 04:26:49,989 - 
--- Run 2 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 86: early stopping
Restoring model weights from the end of the best epoch: 66.


2025-01-09 04:36:32,982 - Validation loss: 0.00595
2025-01-09 04:36:37,037 - 
--- Run 3 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

2025-01-09 09:06:00,642 - Validation loss: 0.00570
2025-01-09 09:06:08,255 - 
--- Run 8 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 50: early stopping
Restoring model weights from the end of the best epoch: 30.


2025-01-09 09:18:15,729 - Validation loss: 0.00548
2025-01-09 09:18:23,157 - 
--- Run 9 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 27.


2025-01-09 09:28:59,867 - Validation loss: 0.00548
2025-01-09 09:29:07,581 - 
--- Run 10 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Completed grid search for input length: 24 Best Validation MSE: 0.00553 Best Hyperparameters: {'units': 64, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 48 Best Validation MSE: 0.00546 Best Hyperparameters: {'units': 256, 'dropout': 0.2, 'learning_rate': 0.01, 'batch_size': 32}

Completed grid search for input length: 72 Best Validation MSE: 0.00531 Best Hyperparameters: {'units': 256, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 128}

Completed grid search for input length: 96 Best Validation MSE: 0.00535 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 120 Best Validation MSE: 0.00528 Best Hyperparameters: {'units': 128, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 144 Best Validation MSE: 0.00528 Best Hyperparameters: {'units': 256, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 168 Best Validation MSE: 0.00526 Best Hyperparameters: {'units': 256, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 192 Best Validation MSE: 0.00524 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 216 Best Validation MSE: 0.00537 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 240 Best Validation MSE: 0.00541 Best Hyperparameters: {'units': 64, 'dropout': 0.4, 'learning_rate': 0.005, 'batch_size': 64}

Completed grid search for input length: 264 Best Validation MSE: 0.00546 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 288 Best Validation MSE: 0.00547 Best Hyperparameters: {'units': 64, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 312 Best Validation MSE: 0.00555 Best Hyperparameters: {'units': 32, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 336 Best Validation MSE: 0.00556 Best Hyperparameters: {'units': 256, 'dropout': 0.1, 'learning_rate': 0.01, 'batch_size': 64}

Completed grid search for input length: 360 Best Validation MSE: 0.00570 Best Hyperparameters: {'units': 32, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 384 Best Validation MSE: 0.00574 Best Hyperparameters: {'units': 256, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 408 Best Validation MSE: 0.00565 Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 432 Best Validation MSE: 0.00568 Best Hyperparameters: {'units': 128, 'dropout': 0.4, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 456 Best Validation MSE: 0.00572 Best Hyperparameters: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}

Completed grid search for input length: 480 Best Validation MSE: 0.00585 Best Hyperparameters: {'units': 256, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 64}

Completed grid search for input length: 504 Best Validation MSE: 0.00596 Best Hyperparameters: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}

### Recalculate the evaluation metrics of length 336 and 48

In [22]:
mean_metrics_list=[]
#for length in best_hyperparameters_improved_model.keys():
for length in [336,48]:
    logging.info(length)
    
    # Number of runs
    n_runs = 10
    
    # get the best hyperparameter of each length
    #hyperparams = best_hyperparameters_improved_model[length]
    hyperparams = best_hyperparameters_improved_model_lstm[length]
    
    # Initialize lists to store mean metrics and all metrics from each run
    mean_metrics_rows = []
    df_all_metrics_list = []
    successful_runs = []  # To track runs with valid mse

    for run in range(1, n_runs + 1):
        logging.info(f"\n--- Run {run} ---")

        # Optionally set a unique seed for each run to ensure variability
        seed = run
        # Train the model
        model, mse = train_model(hyperparams, data_dict, length, seed=seed)
        
        if not np.isnan(mse):  # Only proceed if mse is valid

            if mse>0.007:
                logging.info(f"Run {run} has mse={mse}, skipping.")
            else:
                successful_runs.append(run)
            
                X_train = data_dict[length]['X_train']
                y_train = data_dict[length]['y_train']
                X_val = data_dict[length]['X_val']
                y_val = data_dict[length]['y_val']
                X_test = data_dict[length]['X_test']
                y_test = data_dict[length]['y_test']
    
                #get the true flow and predicted flow
                y_pred_train, y_obs_train = make_prediction(model, X_train, y_train)
                y_pred_val, y_obs_val = make_prediction(model, X_val, y_val)
                y_pred_test, y_obs_test = make_prediction(model, X_test, y_test)
    
                #calculate the evaluation metrics of each output step
                df_train = evaluation(y_pred_train, y_obs_train).add_suffix('_train')
                df_val = evaluation(y_pred_val, y_obs_val).add_suffix('_val')
                df_test = evaluation(y_pred_test, y_obs_test).add_suffix('_test')
    
                df_all_metrics = pd.concat([df_train, df_val, df_test], axis=1)
                df_all_metrics.index.name = length
    
                # Append df_all_metrics to the list
                df_all_metrics_list.append(df_all_metrics)
    
                # Calculate mean for all output step
                #mean_metrics = df_val.mean()
                mean_metrics = pd.concat([df_train.mean(), df_val.mean(), df_test.mean()])
                mean_metrics_row = pd.DataFrame(mean_metrics).T
                mean_metrics_row['MSE_val(loss)'] = mse
                mean_metrics_row['input_len'] = length
                #mean_metrics_row = mean_metrics_row[['input_len','MSE_val(loss)', 'MAE_val', 'RMSE_val', 'MAPE (%)_val']]
                mean_metrics_row = mean_metrics_row[['input_len', 'MSE_val(loss)', 'MAE_train', 'RMSE_train', 'MAPE (%)_train',
                                                 'MAE_val', 'RMSE_val', 'MAPE (%)_val',
                                                 'MAE_test', 'RMSE_test', 'MAPE (%)_test']]
    
                # Append to the list
                mean_metrics_rows.append(mean_metrics_row)
        else:
            logging.info(f"Run {run} has mse=NaN, skipping.")
            
    # Check if there are successful runs before proceeding
    if successful_runs:       
        # Concatenate all df_all_metrics into a single DataFrame with a new level for runs
        concatenated_all_metrics = pd.concat(df_all_metrics_list, keys=successful_runs, names=['Run', 'Time Step'])

        # Calculate the mean across runs for each metric and time step
        # This will group by 'Time Step' and calculate the mean of each metric across all runs
        aggregated_all_metrics_mean = concatenated_all_metrics.groupby('Time Step').mean()
        aggregated_all_metrics_mean.index.name = length

        logging.info("\n--- Aggregated Mean of All Metrics Across 10(Successful) Runs ---")
        # Convert DataFrame to string
        aggregated_all_metrics_mean_str = aggregated_all_metrics_mean.to_string(index=False)    
        # Log the DataFrame
        logging.info("\n" + aggregated_all_metrics_mean_str)
        display(aggregated_all_metrics_mean)
    else:
        logging.info(f"No successful runs for input length {length}.")

    # Check if mean metrics were calculated
    if mean_metrics_rows:        
        # After all runs, create a DataFrame of mean metrics
        mean_metrics_df = pd.concat(mean_metrics_rows, ignore_index=True)
        # Calculate the mean of each metric across the 10 runs
        final_mean_metrics = mean_metrics_df.mean()

        # Create a DataFrame for the final mean metrics
        final_mean_metrics_df = pd.DataFrame(final_mean_metrics).T

        mean_metrics_list.append(final_mean_metrics_df)
    else:
        logging.info(f"No mean metrics calculated for input length {length}.")

mean_metrics_df = pd.concat(mean_metrics_list).reset_index(drop=True)
logging.info("\n--- Final Mean Metrics Across 10 Runs ---")
# Convert DataFrame to string
mean_metrics_df_str = mean_metrics_df.to_string(index=False)    
# Log the DataFrame
logging.info("\n" + mean_metrics_df_str)
mean_metrics_df

2025-01-13 03:13:04,999 - 336
2025-01-13 03:13:05,002 - 
--- Run 1 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 24: early stopping
Restoring model weights from the end of the best epoch: 4.


2025-01-13 03:17:00,921 - Validation loss: 0.02516
2025-01-13 03:17:00,924 - Run 1 has mse=0.025159712880849838, skipping.
2025-01-13 03:17:00,925 - 
--- Run 2 ---


Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 27.


2025-01-13 03:24:43,796 - Validation loss: 0.00592
2025-01-13 03:24:54,346 - 
--- Run 3 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Epoch 41: early stopping
Restoring model weights from the end of the best epoch: 21.


2025-01-13 05:47:27,315 - Validation loss: 0.00574
2025-01-13 05:47:33,103 - 
--- Aggregated Mean of All Metrics Across 10(Successful) Runs ---
2025-01-13 05:47:33,106 - 
 MAE_train  RMSE_train  MAPE (%)_train   MAE_val  RMSE_val  MAPE (%)_val  MAE_test  RMSE_test  MAPE (%)_test
 22.803459   32.110652    7.577846e+10 20.646787 32.056396     37.303585 23.005045  33.472334      26.451181
 25.319244   35.661641    1.134349e+11 23.339565 36.761639     42.026295 26.884089  39.335143      30.953135
 26.278163   37.153974    1.450604e+11 24.232111 38.727691     43.544529 28.543338  41.877545      33.327479
 27.073762   37.887404    1.751645e+11 25.229256 39.809423     49.778754 29.707718  43.337379      37.810941
 28.000700   39.036776    1.986458e+11 26.095277 40.854141     55.257476 30.785630  44.917477      41.768766
 28.939585   40.100789    2.186869e+11 26.985615 41.805527     61.243478 31.592956  46.126067      44.771984


,MAE_train,RMSE_train,MAPE (%)_train,MAE_val,RMSE_val,MAPE (%)_val,MAE_test,RMSE_test,MAPE (%)_test
48,,,,,,,,,
1,22.803459,32.110652,7.577846e+10,20.646787,32.056396,37.303585,23.005045,33.472334,26.451181
2,25.319244,35.661641,1.134349e+11,23.339565,36.761639,42.026295,26.884089,39.335143,30.953135
3,26.278163,37.153974,1.450604e+11,24.232111,38.727691,43.544529,28.543338,41.877545,33.327479
4,27.073762,37.887404,1.751645e+11,25.229256,39.809423,49.778754,29.707718,43.337379,37.810941
5,28.000700,39.036776,1.986458e+11,26.095277,40.854141,55.257476,30.785630,44.917477,41.768766
6,28.939585,40.100789,2.186869e+11,26.985615,41.805527,61.243478,31.592956,46.126067,44.771984


2025-01-13 05:47:33,116 - 
--- Final Mean Metrics Across 10 Runs ---
2025-01-13 05:47:33,120 - 
 input_len  MSE_val(loss)  MAE_train  RMSE_train  MAPE (%)_train   MAE_val  RMSE_val  MAPE (%)_val  MAE_test  RMSE_test  MAPE (%)_test
     336.0       0.006057  25.657465   35.649758    1.868045e+11 24.441407 39.557585     44.706346 29.339979  43.807638      37.848573
      48.0       0.005691  26.402485   36.991873    1.544618e+11 24.421435 38.335803     48.192353 28.419796  41.510991      35.847248


,input_len,MSE_val(loss),MAE_train,RMSE_train,MAPE (%)_train,MAE_val,RMSE_val,MAPE (%)_val,MAE_test,RMSE_test,MAPE (%)_test
0,336.0,0.006057,25.657465,35.649758,1.868045e+11,24.441407,39.557585,44.706346,29.339979,43.807638,37.848573
1,48.0,0.005691,26.402485,36.991873,1.544618e+11,24.421435,38.335803,48.192353,28.419796,41.510991,35.847248


## 8.Calculate the training time and inference time

In [3]:
import time

In [6]:
# Record start time
start_time = time.time()

a=1+2+3+5+6+7+8+9+10

# Record end time and compute the inference time
inference_time = time.time() - start_time
print(f"\nInference time: {inference_time:.6f} seconds")


Inference time: 0.000216 seconds


In [21]:
continued_input_lengths = [24 * i for i in range(11, 22)]
continued_input_lengths

[264, 288, 312, 336, 360, 384, 408, 432, 456, 480, 504]

In [ ]:
# List to collect average inference times for each length
inference_results = []
training_results = []

#for length in best_hyperparameters_improved_model.keys():
for length in continued_input_lengths:
    logging.info(length)
    
    # Number of runs
    n_runs = 10
    
    # get the best hyperparameter of each length
    #hyperparams = best_hyperparameters_improved_model[length]
    hyperparams = best_hyperparameters_improved_model_lstm[length]

    # List to store inference times for each run
    inference_times = []
    training_times = []

    for run in range(1, n_runs + 1):
        logging.info(f"\n--- Run {run} ---")

        # Optionally set a unique seed for each run to ensure variability
        seed = run
        # Train the model
        model, mse, run_training_time = train_model(hyperparams, data_dict, length, seed=seed)
        
        if not np.isnan(mse):  # Only proceed if mse is valid

            if mse>0.007:
                logging.info(f"Run {run} has mse={mse}, skipping.")
            else:
                training_times.append(run_training_time)
            
                X_test = data_dict[length]['X_test']
                y_test = data_dict[length]['y_test']
    
                y_pred_test, y_obs_test, run_inference_time = make_prediction(model, X_test, y_test)

                inference_times.append(run_inference_time)
    
        else:
            logging.info(f"Run {run} has mse=NaN, skipping.")
            
    # Calculate the average inference time across all runs
    avg_inference_time = sum(inference_times) / len(inference_times)
    logging.info(f"\nAverage inference time for {n_runs} runs: {avg_inference_time:.6f} seconds")
    # Compute the average training time over the 10 runs for the current length
    avg_training_time = sum(training_times) / len(training_times)
    logging.info(f"\nAverage training time for length {length}: {avg_training_time:.6f} seconds over {n_runs} runs")

    inference_results.append({
        "length": length,
        "avg_inference_time": avg_inference_time
    })
    training_results.append({
        "length": length,
        "avg_training_time": avg_training_time
    })

# Convert the results into a DataFrame
df_inference = pd.DataFrame(inference_results)
df_training = pd.DataFrame(training_results)
df_inference_training = pd.merge(df_training, df_inference, on='length')
logging.info("\n" + df_inference_training.to_string(index=False))

2025-02-21 12:23:47,110 - 264
2025-02-21 12:23:47,113 - 
--- Run 1 ---
2025-02-21 12:23:47.285839: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22263 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 276f:00:00.0, compute capability: 8.0
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-02-21 12:23:50.485809: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


Epoch 50: early stopping
Restoring model weights from the end of the best epoch: 30.


2025-02-21 12:35:38,643 - Validation loss: 0.00564
2025-02-21 12:35:38,644 - Average training time per epoch: 14.204988 seconds over 50 epochs
2025-02-21 12:35:40,306 - 
Inference time: 1.661437 seconds
2025-02-21 12:35:40,308 - 
--- Run 2 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 59: early stopping
Restoring model weights from the end of the best epoch: 39.


2025-02-21 12:49:07,900 - Validation loss: 0.00548
2025-02-21 12:49:07,902 - Average training time per epoch: 13.683121 seconds over 59 epochs
2025-02-21 12:49:09,539 - 
Inference time: 1.636056 seconds
2025-02-21 12:49:09,541 - 
--- Run 3 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

2025-02-21 16:51:05,760 - Validation loss: 0.00568
2025-02-21 16:51:05,761 - Average training time per epoch: 11.890385 seconds over 76 epochs
2025-02-21 16:51:07,131 - 
Inference time: 1.369198 seconds
2025-02-21 16:51:07,132 - 
--- Run 6 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 81: early stopping
Restoring model weights from the end of the best epoch: 61.


2025-02-21 17:06:10,800 - Validation loss: 0.00571
2025-02-21 17:06:10,801 - Average training time per epoch: 11.153589 seconds over 81 epochs
2025-02-21 17:06:12,189 - 
Inference time: 1.387824 seconds
2025-02-21 17:06:12,191 - 
--- Run 7 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 92: early stopping
Restoring model weights from the end of the best epoch: 72.


2025-02-21 17:26:47,483 - Validation loss: 0.00574
2025-02-21 17:26:47,484 - Average training time per epoch: 13.424185 seconds over 92 epochs
2025-02-21 17:26:49,356 - 
Inference time: 1.871842 seconds
2025-02-21 17:26:49,359 - 
--- Run 8 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [36]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [31]:
inference_results

[{'length': 24, 'avg_inference_time': 0.7298144340515137},
 {'length': 48, 'avg_inference_time': 0.762134850025177},
 {'length': 72, 'avg_inference_time': 1.0686572074890137},
 {'length': 96, 'avg_inference_time': 1.1139964818954469},
 {'length': 120, 'avg_inference_time': 1.1129759788513183},
 {'length': 144, 'avg_inference_time': 1.2348714828491212},
 {'length': 168, 'avg_inference_time': 1.6627337694168092},
 {'length': 192, 'avg_inference_time': 1.7837560176849365},
 {'length': 216, 'avg_inference_time': 1.9610406875610351},
 {'length': 240, 'avg_inference_time': 1.8503576993942261}]

[{'length': 24, 'avg_inference_time': 0.7298144340515137},
 {'length': 48, 'avg_inference_time': 0.762134850025177},
 {'length': 72, 'avg_inference_time': 1.0686572074890137},
 {'length': 96, 'avg_inference_time': 1.1139964818954469},
 {'length': 120, 'avg_inference_time': 1.1129759788513183},
 {'length': 144, 'avg_inference_time': 1.2348714828491212},
 {'length': 168, 'avg_inference_time': 1.6627337694168092},
 {'length': 192, 'avg_inference_time': 1.7837560176849365},
 {'length': 216, 'avg_inference_time': 1.9610406875610351},
 {'length': 240, 'avg_inference_time': 1.8503576993942261}]

In [32]:
training_results

[{'length': 24, 'avg_training_time': 5.195382495249897},
 {'length': 48, 'avg_training_time': 6.4515577810285905},
 {'length': 72, 'avg_training_time': 2.459479804264511},
 {'length': 96, 'avg_training_time': 8.707080613753092},
 {'length': 120, 'avg_training_time': 4.741429529660926},
 {'length': 144, 'avg_training_time': 6.037351604498803},
 {'length': 168, 'avg_training_time': 7.9808274351509265},
 {'length': 192, 'avg_training_time': 16.79577632387864},
 {'length': 216, 'avg_training_time': 19.556736691513613},
 {'length': 240, 'avg_training_time': 10.042025488867797}]

[{'length': 24, 'avg_training_time': 5.195382495249897},
 {'length': 48, 'avg_training_time': 6.4515577810285905},
 {'length': 72, 'avg_training_time': 2.459479804264511},
 {'length': 96, 'avg_training_time': 8.707080613753092},
 {'length': 120, 'avg_training_time': 4.741429529660926},
 {'length': 144, 'avg_training_time': 6.037351604498803},
 {'length': 168, 'avg_training_time': 7.9808274351509265},
 {'length': 192, 'avg_training_time': 16.79577632387864},
 {'length': 216, 'avg_training_time': 19.556736691513613},
 {'length': 240, 'avg_training_time': 10.042025488867797}]

## 11. Calculate variance within runs and steps

### 11.1 Get metrics of all input length, runs and steps

In [21]:
import time

In [22]:
import logging

logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format='%(asctime)s - %(message)s',  # Customize the log message format
    handlers=[
        logging.FileHandler('lstm2_metrics.log'),  # Log messages to 'output.log'
        logging.StreamHandler()             # Also output to console/notebook
    ],
    force=True
)

In [ ]:
all_len_metrics_list=[]
for length in best_hyperparameters_improved_model_lstm.keys():
    logging.info(length)
    
    # Number of runs
    n_runs = 10
    
    # get the best hyperparameter of each length
    # hyperparams = best_hyperparameters_improved_model[length] #after covid-19
    hyperparams = best_hyperparameters_improved_model_lstm[length] #entire data
    # Initialize lists to store mean metrics and all metrics from each run
    mean_metrics_rows = []
    df_each_run_list = []
    successful_runs = []  # To track runs with valid mse

    for run in range(1, n_runs + 1):
        logging.info(f"\n--- Run {run} ---")

        # Optionally set a unique seed for each run to ensure variability
        seed = run
        # Train the model
        model, mse, run_training_time = train_model(hyperparams, data_dict, length, seed=seed)
        
        if not np.isnan(mse):  # Only proceed if mse is valid

            if mse>0.007:
                logging.info(f"Run {run} has mse={mse}, skipping.")
            else:
                successful_runs.append(run)
            
                X_test = data_dict[length]['X_test']
                y_test = data_dict[length]['y_test']
    
                #get the true flow and predicted flow
                y_pred_test, y_obs_test, run_inference_time = make_prediction(model, X_test, y_test)
    
                #calculate the evaluation metrics of each output step
                df_each_run = evaluation(y_pred_test, y_obs_test).add_suffix('_test')
    
                #add a column to show prediction step
                df_each_run.reset_index(inplace=True)  # moves 'Time Step' back to a column
                df_each_run.rename(columns={'Time Step': 'pre_step'}, inplace=True)
    
                #add columns of input length and run step
                df_each_run.insert(0, "input_len", length)
                df_each_run.insert(1, "run_num", run)
    
                # Append df_each_run to the list
                df_each_run_list.append(df_each_run)
        else:
            logging.info(f"Run {run} has mse=NaN, skipping.")
            
    # Check if there are successful runs before proceeding
    if successful_runs:

        concatenated_all_runs = pd.concat(df_each_run_list, ignore_index=True)

        logging.info("\n--- All Metrics Across 10 Runs ---")
        # Convert DataFrame to string
        concatenated_all_runs_str = concatenated_all_runs.to_string(index=False)    
        # Log the DataFrame
        logging.info("\n" + concatenated_all_runs_str)
        display(concatenated_all_runs)
    
        all_len_metrics_list.append(concatenated_all_runs)
    else:
        logging.info(f"No successful runs for input length {length}.")

    all_metrics_df = pd.concat(all_len_metrics_list).reset_index(drop=True)
    logging.info("\n--- Final Mean Metrics Across 10 Runs ---")
    # Convert DataFrame to string
    all_metrics_df_str = all_metrics_df.to_string(index=False)    
    # Log the DataFrame
    logging.info("\n" + all_metrics_df_str)
    display(all_metrics_df)
    all_metrics_df.to_csv("all_metrics_lstm_entire_data.csv", index=False)

2025-10-01 16:10:19,244 - 24
2025-10-01 16:10:19,245 - 
--- Run 1 ---
2025-10-01 16:10:19.503941: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22395 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 276f:00:00.0, compute capability: 8.0
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-10-01 16:10:22.751025: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


Epoch 72: early stopping
Restoring model weights from the end of the best epoch: 52.


2025-10-01 16:20:10,738 - Validation loss: 0.00593
2025-10-01 16:20:10,739 - Average training time per epoch: 8.197825 seconds over 72 epochs
2025-10-01 16:20:11,621 - 
Inference time: 0.880594 seconds
2025-10-01 16:20:11,632 - 
--- Run 2 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 86: early stopping
Restoring model weights from the end of the best epoch: 66.


2025-10-01 16:31:47,326 - Validation loss: 0.00595
2025-10-01 16:31:47,328 - Average training time per epoch: 8.085874 seconds over 86 epochs
2025-10-01 16:31:48,265 - 
Inference time: 0.935209 seconds
2025-10-01 16:31:48,274 - 
--- Run 3 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

2025-10-02 17:02:52,083 - Validation loss: 0.00621
2025-10-02 17:02:52,085 - Average training time per epoch: 17.814813 seconds over 44 epochs
2025-10-02 17:02:53,792 - 
Inference time: 1.706467 seconds
2025-10-02 17:02:53,800 - 
--- All Metrics Across 10 Runs ---
2025-10-02 17:02:53,803 - 
 input_len  run_num  pre_step  MAE_test  RMSE_test  MAPE (%)_test
       360        1         1 23.530288  34.340162      27.940612
       360        1         2 26.707869  39.323072      32.694042
       360        1         3 28.306820  41.772229      36.055575
       360        1         4 29.434195  43.629459      38.534906
       360        1         5 30.648262  45.509419      44.491176
       360        1         6 31.525950  47.070622      47.947680
       360        2         1 23.445282  34.302103      27.312893
       360        2         2 26.713183  39.030972      30.314955
       360        2         3 28.317860  41.121517      33.991365
       360        2         4 28.983029  42.5152

,input_len,run_num,pre_step,MAE_test,RMSE_test,MAPE (%)_test
0,360,1,1,23.530288,34.340162,27.940612
1,360,1,2,26.707869,39.323072,32.694042
2,360,1,3,28.306820,41.772229,36.055575
3,360,1,4,29.434195,43.629459,38.534906
4,360,1,5,30.648262,45.509419,44.491176
5,360,1,6,31.525950,47.070622,47.947680
6,360,2,1,23.445282,34.302103,27.312893
7,360,2,2,26.713183,39.030972,30.314955
8,360,2,3,28.317860,41.121517,33.991365
9,360,2,4,28.983029,42.515232,35.144119


2025-10-02 17:02:53,812 - 
--- Final Mean Metrics Across 10 Runs ---
2025-10-02 17:02:53,829 - 
 input_len  run_num  pre_step  MAE_test  RMSE_test  MAPE (%)_test
        24        1         1 23.117895  33.934771      26.287765
        24        1         2 26.752603  39.054002      32.630426
        24        1         3 28.556700  41.681333      35.396921
        24        1         4 29.726387  43.305771      39.442665
        24        1         5 30.883198  44.730370      43.691297
        24        1         6 31.451810  45.994106      44.415696
        24        2         1 23.137344  33.552856      27.681585
        24        2         2 26.854843  39.163988      33.407559
        24        2         3 29.128310  42.652937      34.382188
        24        2         4 30.274011  44.388824      36.881103
        24        2         5 31.314692  45.603170      42.133644
        24        2         6 32.389892  47.333435      44.472131
        24        3         1 22.682276  33.24

,input_len,run_num,pre_step,MAE_test,RMSE_test,MAPE (%)_test
0,24,1,1,23.117895,33.934771,26.287765
1,24,1,2,26.752603,39.054002,32.630426
2,24,1,3,28.556700,41.681333,35.396921
3,24,1,4,29.726387,43.305771,39.442665
4,24,1,5,30.883198,44.730370,43.691297
...,...,...,...,...,...,...
865,360,10,2,27.358514,39.622706,35.421863
866,360,10,3,29.235388,42.238577,39.891573
867,360,10,4,30.627051,44.096759,43.604772
868,360,10,5,31.760118,45.676133,48.045818


2025-10-02 17:02:53,841 - 384
2025-10-02 17:02:53,843 - 
--- Run 1 ---
/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
all_metrics_df.to_csv("all_metrics_lstm_entire_data.csv", index=False)